# This notebooks shows some basic operations for Quantum Computing

ProjectQ must be installed before you execute it.

### 1. Import needed modules

In [1]:
from projectq.ops import Ph,H,Measure,X,All
from projectq.meta import Control
from projectq.cengines import MainEngine
import numpy as np
np.set_printoptions(precision=3)

This is a helper function to print a n-qubit state. **Just, execute it**

In [2]:
def get_state_as_str(eng,qubits,cheat=False,ancilla=True):
    import numpy as np
    s=""
    if (cheat):
        print("Cheat: ", eng.backend.cheat())
    if (len(qubits)==1):
        for i in range(2):
            #print("bits:%d%s"%(i,bits))
            a=eng.backend.get_amplitude("%d"%(i),qubits)
            if (a.real!=0)|(a.imag!=0):
                if s!="":
                    s=s+"+"
                a="({:.2f})".format(a)
                s=s+"%s|%d>"%(a,i)

    else:
        for j in range(2**(len(qubits)-1)):
            bits=np.binary_repr(j,width=len(qubits)-1)
            #print("Bits:",j,bits)
            for i in range(2):
                #print("bits:%d%s"%(i,bits))
                a=eng.backend.get_amplitude("%d%s"%(i,bits[-1::-1]),qubits)
                if (a.real!=0)|(a.imag!=0):
                    if s!="":
                        s=s+"+"
                    a="({:.2f})".format(a)
                    if (ancilla):
                        s=s+"%s|%s>|%d>"%(a,bits,i)
                    else:
                        s=s+"%s|%s%d>"%(a,bits,i)
                #print(s)
   
    
    return(s)

# Qubit order

When you make an allocation of a quantum register, take into account the order. Lower index of the Quantum Register is the lower bit of the binary representation of a quantum state. For example:

1. Allocate a quantum register q with 3 qubits
2. Apply a X gate to the first one using X | q[0]
3. Check the final state, converting it to a number using binary representation, so $|001\rangle = |1\rangle$ and $|010\rangle=|2\rangle$

### 1.Start the Engine

In [3]:
eng=MainEngine()

### 2. Initialise the quantum state
Check the vector of the quantum state and its binary representation. 

**Remember**: in python, the index of a vector of length N, starts on 0 and ends on N-1. 

In [4]:
q=eng.allocate_qureg(3)
X | q[2]
eng.flush()
get_state_as_str(eng, q,cheat=True,ancilla=False)

Cheat:  ({0: 0, 1: 1, 2: 2}, [0j, 0j, 0j, 0j, (1+0j), 0j, 0j, 0j])


'(1.00+0.00j)|100>'

### 3.Do not forget to delete the Engine

In [5]:
All(Measure) | q
eng.flush()

# Phase Kickback

When a qubit that has been initialized to a Walsh-Hadamard state $\frac{1}{\sqrt{2}}(|0\rangle+|1\rangle)$ is used as a control bit for a phase gate $U(\phi)$ on a state $|\Psi\rangle$, the result is the final state is $\frac{1}{\sqrt{2}}(|0\rangle>+e^{i\phi}|1\rangle)\otimes|\Psi>$. So, the fase is transferred to the control qubit. 

<img src="Images/CPhase.png"/>

This exercise will help you to check it experimentally. Remember, for this exercise, qubit **q0** is the top one of this picture, so, Phase Kickback goes to qubit **q0**


### 1.Start the Engine

In [6]:
eng=MainEngine()

### 2. Allocate two qubits: q1 and q2

In [7]:
q0=eng.allocate_qubit()
q1=eng.allocate_qubit()

### 3. Apply a Hadamard gate to the first qubit

In [8]:
H|q0
eng.flush()
get_state_as_str(eng, q0+q1,cheat=False)

'(0.71+0.00j)|0>|0>+(0.71+0.00j)|0>|1>'

### 4. Apply a controlled phase gate to the second qubit, controlled by the first qubit

The Phase gate has a matrix as $e^{i\phi}I$. Let check which is the generated matrix for a rotation $\phi=\pi/4$

In [9]:
import math
pi=math.pi
a=Ph(pi/4).matrix
print(a)


[[0.707+0.707j 0.   +0.j   ]
 [0.   +0.j    0.707+0.707j]]


Ok. Apply the Controlled Phase gate for $\phi=\pi/4$ to the second qubit

In [10]:
with Control(eng,q0):
    Ph(pi/4)|q1

And flush the current circuit to calculate the result state

In [11]:
eng.flush()

### 5.Print the result state. 

In [12]:
get_state_as_str(eng, q0+q1,cheat=True)

Cheat:  ({0: 0, 1: 1}, [(0.7071067811865475+0j), (0.5000000000002242+0.4999999999997758j), 0j, 0j])


'(0.71+0.00j)|0>|0>+(0.50+0.50j)|0>|1>'

In [13]:
Measure | q0
Measure | q1
eng.flush()

In [14]:
del eng

# Toffoli Gate

<img src="Images/Toffoli.png"/>
This gate is know to be a universal gate. It uses two contol qubits to produce as output 

<table>
    <tr><td colspan="3">INPUT</td><td colspan="3">OUTPUT</td></tr>
    <tr><td>0</td><td>0</td><td>0</td><td>0</td><td>0</td><td>0</td></tr>
    <tr><td>0</td><td>0</td><td>1</td><td>0</td><td>0</td><td>1</td></tr>
    <tr><td>0</td><td>1</td><td>0</td><td>0</td><td>1</td><td>0</td></tr>
    <tr><td>0</td><td>1</td><td>1</td><td>0</td><td>1</td><td>1</td></tr>
    <tr><td>1</td><td>0</td><td>0</td><td>1</td><td>0</td><td>0</td></tr>
    <tr><td>1</td><td>0</td><td>1</td><td>1</td><td>0</td><td>1</td></tr>
    <tr><td>1</td><td>1</td><td>0</td><td>1</td><td>1</td><td>1</td></tr>
    <tr><td>1</td><td>1</td><td>1</td><td>1</td><td>1</td><td>0</td></tr>
            
</table>

The Matrix representation is:
<math>
\begin{bmatrix}
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
\end{bmatrix}
</math>

It can be also described as mapping bits {a, b, c} to {a, b, c XOR (a AND b)}.

In [15]:
eng=MainEngine()

### 1.Create the quantum registers, one for each qubit

In [16]:
ancilla=eng.allocate_qubit()
c1=eng.allocate_qubit()
c2=eng.allocate_qubit()


### 2.Initiallize the qubits to check the main cases. For example, to initialize the last row of the table, you should apply one X gate to each qubit

In [17]:
X|c1
X|c2
X|ancilla

### 3.Apply now the Controlled-Controlled-X gate. To do it, use the meta operation Control

In [18]:
with Control(eng,c1):
    with Control(eng,c2):
        X|ancilla

### 4.Execute the circuit to get the final state.

In [19]:
eng.flush()

### 5. Print the result state

If you want to execute another case, remenber to delete the Engine before going back to step 1

In [20]:
get_state_as_str(eng, ancilla+c1+c2)

'(1.00+0.00j)|11>|0>'

In [21]:
All(Measure) | c1+c2+ancilla
del eng

### 6.In ProjectQ you can create a new controlled gate of any number of qubits using the special operation ControlledGate({gate to control},{number of control qubits})

In [22]:
from projectq.ops import ControlledGate
Toffoli=ControlledGate(X, 2) 

In [23]:
eng=MainEngine()
ancilla=eng.allocate_qubit()
c1=eng.allocate_qubit()
c2=eng.allocate_qubit()
X|c1
X|c2
X|ancilla
Toffoli| (c1, c2, ancilla)
eng.flush()
print(get_state_as_str(eng, ancilla+c1+c2,cheat=True))
All(Measure) | c1+c2+ancilla
del eng

Cheat:  ({0: 0, 1: 1, 2: 2}, [0j, 0j, 0j, 0j, 0j, 0j, (1+0j), 0j])
(1.00+0.00j)|11>|0>


### 7. or use the predefined builtin Toffoli gate

In [24]:
from projectq.ops import Toffoli 

In [25]:
eng=MainEngine()
c1=eng.allocate_qubit()
c2=eng.allocate_qubit()
ancilla=eng.allocate_qubit()
X|c1
X|c2
X|ancilla

Toffoli| (c1, c2, ancilla)

eng.flush()
print(get_state_as_str(eng, ancilla+c2+c1))
All(Measure) | c1+c2+ancilla
del eng

(1.00+0.00j)|11>|0>


# Multiple Controlled k-Gates

In general, in a register with n+k qubits a n-Controlled U acting on k-qubits is defined as:

$C^n(U)|x_1 x_2 \dots x_n\rangle|\Psi\rangle = |x_1 x_2 \dots x_n\rangle U^{x_1 x_2 \dots x_n}|\Psi\rangle$

For example, the Tofooli's gate is defined as:

$C^n(X)|x_1 x_2\rangle>|y_1\rangle = |x_1 x_2\rangle X^{x_1 x_2}|y_1\rangle$

If $x_1=1,x_2=1$, then $C^n(X)|11\rangle|0\rangle = |11\rangle X^{1x1}|0\rangle=|11\rangle X|0\rangle=|11\rangle|1\rangle$

So, check that this is true **H** with three control qubits. Check the 3-Controlled gate $C^3(H)$


In [26]:
eng=MainEngine()

In [27]:
a=eng.allocate_qubit(1)
C=eng.allocate_qureg(3)


Funtion to initialize the control qubits to a bit mask

In [28]:
def init_control(C,bits):
    for i in range(len(bits)):      
        if bits[i]=="1":
            X|C[-1-i]

Initialize the control bits as a string containitn {1,0} for each qubit. Which is the result for **"011"**? And for **"111"**?

In [29]:
init_control(C,"100")
eng.flush()
get_state_as_str(eng, a+C)

'(1.00+0.00j)|100>|0>'

Create the controlled gate

In [30]:
def CnGate(eng,C,G,a):
    if (len(C)>1):
        with Control(eng,C[0]):
            CnGate(eng,C[1:],G,a)
    else:
        with Control(eng,C):
            G|a

In [31]:
CnGate(eng,C,H,a)
eng.flush()

In [32]:
get_state_as_str(eng, a+C)

'(1.00+0.00j)|100>|0>'

In [33]:
All(Measure) | C+a
del eng

### Or using the ControlledGate operation defined previously

In [34]:
CH=ControlledGate(H,3)

In [35]:
eng=MainEngine()
a=eng.allocate_qubit(1)
C=eng.allocate_qureg(3)

init_control(C,"111")
eng.flush()
print(get_state_as_str(eng, a+C))

CH|(C,a)
eng.flush()
print(get_state_as_str(eng, a+C))

All(Measure) | C+a
del eng

(1.00+0.00j)|111>|0>
(0.71+0.00j)|111>|0>+(0.71+0.00j)|111>|1>


Now you can check it for any other number of qubits and unitary gates.